## LSTM Changing Batch Size for Training and Testing

* When using built-in method of keras, the batch size limits the number of samples to be shown to the network before a weight update can be performed. Specifically, the batch size used when fitting your model controls how many predictions you must make at a time.
* This will become an error when the number of predictions is lower than the batch size. For example, you may get the best results with a large batch size, but are required to make predictions for one observation at a time on something like a time series or sequence problem.
* So, it will be better to have different batch size for training and testing.


* Reference: https://machinelearningmastery.com/use-different-batch-sizes-training-predicting-python-keras/

In [31]:
from tensorflow import set_random_seed
set_random_seed(410)

from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential

import pandas as pd

In [32]:
# Generate data
## create sequence
length = 10
sequence = [i/float(length) for i in range(length)]
print sequence

## create X/y pairs
df = pd.DataFrame(sequence)
df = pd.concat([df, df.shift(1)], axis=1)  # add second column which is the first column shift 1 period
df.dropna(inplace=True)
df

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


,0,0
1,0.1,0.0
2,0.2,0.1
3,0.3,0.2
4,0.4,0.3
5,0.5,0.4
6,0.6,0.5
7,0.7,0.6
8,0.8,0.7
9,0.9,0.8


In [33]:
## convert to LSTM friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)  # LSTM needs (number of records, timesteps, number of features)
print(X.shape, y.shape)

((9, 1, 1), (9,))


### 3 Solutions

* If we just want to predict 1 step, there are 3 solutions:
  * Solution 1 - batch size = 1
    * Both training and testing are using batch_size=1.
    * This can have the effect of faster learning, but also adds instability to the learning process as the weights widely vary with each batch.
  * Solution 2 - batch size = n
    * Make all the predictions at once in a batch.
    * But later you need to use all predictions made at once, or only keep the first prediction and discard the rest.
  * Solution 3 - different batch size for training & testing
    * The better solution
    * majorly is to copy the weights from the fit network and to create a new network with the pre-trained weights.

In [40]:
# Solution 3
## Model 1 - 3 batches for training

### configure network
n_batch = 3
n_epoch = 1000
n_neurons = 10

### design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
### fit network
for i in range(n_epoch):
    model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
    model.reset_states() # manually set internal state update after epoch, otherwise state will reset after each batch

Epoch 1/1
9/9 [==============================] - 1s 131ms/step - loss: 0.2290
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.2224
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.2160
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.2096
Epoch 1/1
9/9 [==============================] - 0s 3ms/step - loss: 0.2033
Epoch 1/1
9/9 [==============================] - 0s 3ms/step - loss: 0.1972
Epoch 1/1
9/9 [==============================] - 0s 3ms/step - loss: 0.1913
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.1855
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.1798
Epoch 1/1
9/9 [==============================] - 0s 3ms/step - loss: 0.1743
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.1689
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.1636
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.1584
Epoch 1/1


9/9 [==============================] - 0s 1ms/step - loss: 0.0063
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0063
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0062
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0061
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0061
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0061
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0060
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0060
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0059
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0059
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0058
Epoch 1/1
9/9 [=======

9/9 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0040
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 1/1
9/9 [=======

9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 1/1
9/9 [=======

9/9 [==============================] - 0s 1ms/step - loss: 0.0032
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0032
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0032
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0031
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0031
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0031
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0031
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0031
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 1/1
9/9 [=======

9/9 [==============================] - 0s 1ms/step - loss: 0.0026
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0026
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 1/1
9/9 [=======

9/9 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 1/1
9/9 [=======

9/9 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0014
Epoch 1/1
9/9 [=======

9/9 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 1/1
9/9 [=======

9/9 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
9/9 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
9/9 [=======

In [41]:
## Model 2 - 1 batch for prediction

### re-define the batch size
n_batch = 1

### re-define model
new_model = Sequential()
new_model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
new_model.add(Dense(1))

### copy weights
old_weights = model.get_weights()
new_model.set_weights(old_weights)

### compile model
new_model.compile(loss='mean_squared_error', optimizer='adam')

In [42]:
# Use original data to check the prediction (NOT suggested)
for i in range(len(X)):
    testX, testy = X[i], y[i]
    testX = testX.reshape(1, 1, 1)
    yhat = new_model.predict(testX, batch_size=n_batch)
    print('>Expected=%.1f, Predicted=%.1f' % (testy, yhat))

>Expected=0.0, Predicted=0.1
>Expected=0.1, Predicted=0.2
>Expected=0.2, Predicted=0.4
>Expected=0.3, Predicted=0.7
>Expected=0.4, Predicted=0.8
>Expected=0.5, Predicted=0.9
>Expected=0.6, Predicted=0.9
>Expected=0.7, Predicted=0.9
>Expected=0.8, Predicted=0.9


In [43]:
# Try a new set of data but has the same values as original data
length = 5
sequence = [i/10.0 for i in range(length)]
print sequence

## create X/y pairs
df = pd.DataFrame(sequence)
df = pd.concat([df, df.shift(1)], axis=1)  # add second column which is the first column shift 1 period
df.dropna(inplace=True)
df

[0.0, 0.1, 0.2, 0.3, 0.4]


,0,0
1,0.1,0.0
2,0.2,0.1
3,0.3,0.2
4,0.4,0.3


In [44]:
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)  # LSTM needs (number of records, timesteps, number of features)
print(X.shape, y.shape)

((4, 1, 1), (4,))


In [45]:
# predict
for i in range(len(X)):
    testX, testy = X[i], y[i]
    testX = testX.reshape(1, 1, 1)
    yhat = new_model.predict(testX, batch_size=n_batch)
    print('>Expected=%.1f, Predicted=%.1f' % (testy, yhat))

>Expected=0.0, Predicted=0.9
>Expected=0.1, Predicted=0.9
>Expected=0.2, Predicted=0.9
>Expected=0.3, Predicted=0.9


In [46]:
# Try a new set of data but has the different values as original data
length = 20
sequence = [i/10.0 for i in range(10, length)]
print sequence

## create X/y pairs
df = pd.DataFrame(sequence)
df = pd.concat([df, df.shift(1)], axis=1)  # add second column which is the first column shift 1 period
df.dropna(inplace=True)
df

[1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9]


,0,0
1,1.1,1.0
2,1.2,1.1
3,1.3,1.2
4,1.4,1.3
5,1.5,1.4
6,1.6,1.5
7,1.7,1.6
8,1.8,1.7
9,1.9,1.8


In [47]:
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)  # LSTM needs (number of records, timesteps, number of features)
print(X.shape, y.shape)

((9, 1, 1), (9,))


In [48]:
# predict
for i in range(len(X)):
    testX, testy = X[i], y[i]
    testX = testX.reshape(1, 1, 1)
    yhat = new_model.predict(testX, batch_size=n_batch)
    print('>Expected=%.1f, Predicted=%.1f' % (testy, yhat))

>Expected=1.0, Predicted=1.0
>Expected=1.1, Predicted=1.0
>Expected=1.2, Predicted=1.0
>Expected=1.3, Predicted=1.0
>Expected=1.4, Predicted=1.0
>Expected=1.5, Predicted=1.0
>Expected=1.6, Predicted=1.0
>Expected=1.7, Predicted=1.0
>Expected=1.8, Predicted=1.0


## Summary

* I know, wit epoch=1000, what's shown on github is very long... But 1000 epoch brough better accuracy, although even when using the original data to predict, it dind't look good, I tried different tensorflow seed, the seed will also make a difference. But anyway, 1000 epoch got better prediction...
* Later I tried new datasets to predict
  * When I was using a subset of original data to predict, it predicts results didn't appear in this new dataset
  * I tried a dataset that its values didn't not appear in the original data, and it cannot predict those values
* So, having a dataset large enough to train is the rule of thumb. Also, the training data better contains the testing cases.